In [1]:
import pandas as pd
import re
import difflib

In [164]:
# Movie Info
df_tmdb_movie = pd.read_csv("H:/DataIntegation/tmdb/tmdb.csv")
df_tmdb_movie.head()

,Unnamed: 0,id,title,overview,popularity,vote_average,vote_count,release_date,keywords,genres,cast,crew
0,0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",31.222,8.7,3323,1995-10-20,[],"['Comedy', 'Drama', 'Romance']","['Shah Rukh Khan', 'Kajol', 'Amrish Puri', 'An...",['Aditya Chopra']
1,1,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,76.654,8.7,20434,1994-09-23,"['prison', 'corruption', 'police brutality', '...","['Drama', 'Crime']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",['Frank Darabont']
2,2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",75.306,8.7,15270,1972-03-14,"['italy', 'loss of loved one', 'love at first ...","['Drama', 'Crime']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",['Francis Ford Coppola']
3,3,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,21.501,8.6,1369,2020-07-31,['based on novel or book'],['Romance'],"['Melanie Zanetti', 'Giulio Berruti', 'James A...",['Tosca Musk']
4,4,424,Schindler's List,The true story of how businessman Oskar Schind...,40.585,8.6,12202,1993-11-30,"['based on novel or book', 'factory', 'concent...","['Drama', 'History', 'War']","['Liam Neeson', 'Ben Kingsley', 'Ralph Fiennes...",['Steven Spielberg']


In [ ]:
df_tmdb_movie.info()

In [148]:
# Drop các cột Unnamed:
df_tmdb_movie.drop('Unnamed: 0', axis = 1, inplace = True)

In [150]:
# Drop id, popularity
df_tmdb_movie.drop(['id', 'popularity'], axis = 1, inplace = True)

In [151]:
df_tmdb_movie.head()

,title,overview,vote_average,vote_count,release_date,keywords,genres,cast,crew
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",8.7,3323,1995-10-20,[],"['Comedy', 'Drama', 'Romance']","['Shah Rukh Khan', 'Kajol', 'Amrish Puri', 'An...",['Aditya Chopra']
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,8.7,20434,1994-09-23,"['prison', 'corruption', 'police brutality', '...","['Drama', 'Crime']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",['Frank Darabont']
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",8.7,15270,1972-03-14,"['italy', 'loss of loved one', 'love at first ...","['Drama', 'Crime']","['Marlon Brando', 'Al Pacino', 'James Caan', '...",['Francis Ford Coppola']
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,8.6,1369,2020-07-31,['based on novel or book'],['Romance'],"['Melanie Zanetti', 'Giulio Berruti', 'James A...",['Tosca Musk']
4,Schindler's List,The true story of how businessman Oskar Schind...,8.6,12202,1993-11-30,"['based on novel or book', 'factory', 'concent...","['Drama', 'History', 'War']","['Liam Neeson', 'Ben Kingsley', 'Ralph Fiennes...",['Steven Spielberg']


Kết quả Schema Matching:

- Bảng Movie

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| title         | movie_title           |
| overview      | movie_info            |
| release_date  | release_date          |
| genres        | genres                |
| cast          | actor                 |
| crew          | directors             |

- Bảng Rating: như nhận xét, kết quả matching sai hoàn toàn

Cấu trúc giá trị của genres, crew, cast theo như mô tả trong data warehouse là <genres 1>, <genres 2>, ... , trong khi đó ở trong tmdb cấu trúc của genres lại là [<genres 1>,<genres 2>,...] , đồng thời các thể loại trong MovieLen có thể khác so với thể loại trong Data Warehouse => cần phải biến đổi lại

# Xử lý trường genres

Từ 2 danh sách thể loại, ta có thể mapping như dưới:
|Data warehouse Genres|MovieLen Genres |
| :- | :- |
|‘Classic’||
|‘Documentary’|‘Documentary’|
|‘Western’|‘Western’|
|‘Horror’|‘Horror’|
|‘Science Fiction & Fantasy’|‘Sci-Fi’, ‘Fantasy’, 'Science Fiction'|
|‘Gay & Lesbian’||
|‘Drama’|‘Drama’|
|‘Comedy’|‘Comedy’|
|‘Cult Movies’||
|‘Romance’|‘Romance’|
|‘Television’||
|‘Sport & Fitness’||
|‘Art House & International’||
|‘Special Interest’||
|‘Animation’|‘Animation’|
|‘Musical & Performing Arts’|‘Musical’|
|‘Faith & Spirituality’||
|‘Mystery & Suspense’|‘Mystery’|
|‘Anime & Manga’||
|‘Action & Adventure’|‘Action’, ‘Adventure’|
|‘Kids & Family’|‘Family’|

Những giá trị như TV Movie ta sẽ thêm vào Data Warehouse

In [152]:
# Chuyển string thành list
def genres_preprocess(genres):
    list_genres = ''
    if(len(genres) > 1):
        list_String = genres[1:-1].split(", ")
        for idx, i in enumerate(list_String):
            if(idx == len(list_String)-1):
                list_genres += i[1:-1]
            else:    
                list_genres += i[1:-1] + ','
    
    return list_genres
df_tmdb_movie["genres"]=[genres_preprocess(i) for i in df_tmdb_movie["genres"]]

In [ ]:
df_tmdb_movie["genres"]

In [154]:
# Lấy danh sách các thể loại có trong tmdb
df_tmdb_movie["genres"]=[i.split(",") for i in df_tmdb_movie["genres"]]
tmdb_genres_list = []
for i in df_tmdb_movie["genres"]:
    tmdb_genres_list+=i
tmdb_genres_list = list(set(tmdb_genres_list))
tmdb_genres_list

['Fantasy',
 '',
 'Crime',
 'Western',
 'Horror',
 'Animation',
 'Adventure',
 'Drama',
 'Comedy',
 'TV Movie',
 'Family',
 'History',
 'Mystery',
 'War',
 'Action',
 'Science Fiction',
 'Romance',
 'Music',
 'Thriller']

In [155]:
replace_genres_dict = {'Mystery' : 'Mystery & Suspense', 'Thriller': 'Thrill',
                        'Sci-Fi': 'Science Fiction & Fantasy', 'Fantasy': 'Science Fiction & Fantasy', 'Science Fiction': 'Science Fiction & Fantasy',
                       'Adventure': 'Action & Adventure','Action': 'Action & Adventure',
                       'Music': 'Musical & Performing Arts',
                        'Family': 'Kids & Family'}
def preprocess_genres(genre):
    new_list_genres = ""
    if(genre == ''):
        new_list_genres = None
    else:
        for idx, i in enumerate(genre):
            if i in replace_genres_dict:
                new_list_genres+=replace_genres_dict[i]
            else:
                new_list_genres+=i
            if idx != len(genre)-1:
                new_list_genres+=", "
    return new_list_genres
df_tmdb_movie["genres"] = [preprocess_genres(i) for i in df_tmdb_movie["genres"]]

In [ ]:
df_tmdb_movie["genres"]

In [129]:
# Thêm các giá trị mới vào trong genres warehouse
# Giữ đg dẫn
warehouse_genres_list+=["TV Movie"]
warehouse_genres_list = list(set(warehouse_genres_list))
pd.DataFrame(warehouse_genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

## Xử lý Cast

In [157]:
# Chuyển string thành list
def cast_preprocess(cast):
    list_genres = ''
    if(len(cast) > 1):
        list_String = cast[1:-1].split(", ")
        for idx, i in enumerate(list_String):
            if(idx == len(list_String)-1):
                list_genres += i[1:-1]
            else:    
                list_genres += i[1:-1] + ', '
    
    return list_genres
df_tmdb_movie["cast"]=[cast_preprocess(i) for i in df_tmdb_movie["cast"]]

In [ ]:
df_tmdb_movie["cast"]

## Xử lý trường Crew

In [165]:
# Chuyển string thành list
def cast_preprocess(cast):
    list_genres = ''
    if(len(cast) > 2):
        list_genres = cast[2:-2]
    return list_genres
df_tmdb_movie["crew"]=[cast_preprocess(i) for i in df_tmdb_movie["crew"]]

In [166]:
df_tmdb_movie["crew"]

0              Aditya Chopra
1             Frank Darabont
2       Francis Ford Coppola
3                 Tosca Musk
4           Steven Spielberg
                ...         
9475         Patrice Leconte
9476             Des McAnuff
9477            Chris Fisher
9478     Perry Andelin Blake
9479          Joseph Sargent
Name: crew, Length: 9480, dtype: object

## Lưu dữ liệu

In [ ]:
df_tmdb_movie.to_csv("H:/DataIntegation/tmdb/movies_preprocess.csv", index=False)